In [28]:
import osmnx as ox
from shapely.geometry import Point, LineString, MultiLineString
from shapely.ops import linemerge
import geopandas as gpd
import folium

# 1️⃣ Define the area and street name
area_name = "Hammersmith and Fulham, London, UK"
street_name = "Askew Road"

# 2️⃣ Fetch all roads (features tagged as highways) in the area
roads = ox.features_from_place(area_name, tags={"highway": True})

# 3️⃣ Filter for the specific street name
askew_road = roads[roads["name"] == street_name]

if askew_road.empty:
    raise ValueError(f"No road found named '{street_name}' in '{area_name}'")

# 4️⃣ Extract only LineString geometries
linestrings = []
for geom in askew_road.geometry:
    if geom.geom_type == 'LineString':
        linestrings.append(geom)
    elif geom.geom_type == 'MultiLineString':
        linestrings.extend(list(geom.geoms))

if not linestrings:
    raise ValueError("No LineString geometries found")

# Create a MultiLineString and merge
multi_line = MultiLineString(linestrings)
road_line = linemerge(multi_line)

# If still a MultiLineString, work with the longest segment
if road_line.geom_type == 'MultiLineString':
    print("Warning: Road consists of multiple disconnected segments")
    road_line = max(road_line.geoms, key=lambda x: x.length)
    print(f"Working with longest segment of length {road_line.length:.2f} degrees")

# 5️⃣ Project to UTM for distance-accurate geometry
road_gdf = gpd.GeoDataFrame(geometry=[road_line], crs="EPSG:4326")
road_projected_gdf = road_gdf.to_crs(road_gdf.estimate_utm_crs())
road_projected = road_projected_gdf.geometry.iloc[0]

# 6️⃣ Sample points every 50 m
distance = 50
points = [road_projected.interpolate(d) for d in range(0, int(road_projected.length), distance)]

# 7️⃣ Convert back to latitude/longitude (EPSG:4326)
points_gdf = gpd.GeoDataFrame(geometry=points, crs=road_projected_gdf.crs)
points_latlon_gdf = points_gdf.to_crs("EPSG:4326")

# 8️⃣ Use the converted GeoDataFrame
gdf_points = points_latlon_gdf

# 9️⃣ Create interactive Folium map
center = gdf_points.geometry.iloc[len(gdf_points)//2]
m = folium.Map(location=[center.y, center.x], zoom_start=16)

# 10️⃣ Add the Askew Road line
folium.GeoJson(askew_road.to_crs(epsg=4326),
               name="Askew Road",
               style_function=lambda x: {"color": "blue", "weight": 4}).add_to(m)

# 11️⃣ Add red points every 50 m
for i, point in enumerate(gdf_points.geometry):
    folium.CircleMarker(
        location=[point.y, point.x],
        radius=4,
        color="red",
        fill=True,
        fill_color="red",
        tooltip=f"Point {i} (~{i*50} m)"
    ).add_to(m)

# 12️⃣ Save to file

Working with longest segment of length 0.01 degrees


In [29]:
pth = "askew_road_points_map.html"
m.save(pth)
print(f"Map saved to: {pth}")
print(f"Total road length: {road_projected.length:.0f} meters")
print(f"Number of sample points: {len(gdf_points)}")

Map saved to: askew_road_points_map.html
Total road length: 892 meters
Number of sample points: 18


In [30]:
gdf_points

,geometry
0,POINT (-0.23943 51.50011)
1,POINT (-0.23963 51.50055)
2,POINT (-0.23984 51.50098)
3,POINT (-0.24014 51.50138)
4,POINT (-0.24078 51.50156)
5,POINT (-0.24146 51.50169)
6,POINT (-0.24207 51.50193)
7,POINT (-0.24261 51.50223)
8,POINT (-0.24297 51.50262)
9,POINT (-0.24319 51.50304)
